<a href="https://colab.research.google.com/github/kavyajeetbora/recipe_recommender/blob/master/notebooks/food_recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 18.4 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=99755a846bc6437cecfddbfef3541d8edebca6358502632a4f2514597c8b4861
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import sys
import matplotlib.pyplot as plt
import seaborn as sns
from sentence_transformers import SentenceTransformer

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Read the data
nutrition - calories (#), total fat (PDV), sugar (PDV) , sodium (PDV) , protein (PDV) , saturated fat (PDV) ,carbohydrates (PDV)

In [ ]:
%%time
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Projects/food_recommender/data_file/RAW_recipes.csv')
df.head()

CPU times: user 5.1 s, sys: 1.02 s, total: 6.12 s
Wall time: 6.7 s


,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7
1,a bit different breakfast pizza,31490,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6
2,all in the kitchen chili,112140,130,196586,2005-02-25,"['time-to-make', 'course', 'preparation', 'mai...","[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",6,"['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...",13
3,alouette potatoes,59389,45,68585,2003-04-14,"['60-minutes-or-less', 'time-to-make', 'course...","[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",11,['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...",11
4,amish tomato ketchup for canning,44061,190,41706,2002-10-25,"['weeknight', 'time-to-make', 'course', 'main-...","[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]",5,['mix all ingredients& boil for 2 1 / 2 hours ...,my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'sugar...",8


## Transform the data

In [ ]:
## Split the nutrition column to separate columns
df[['calories','total fat (PDV)','sugar (PDV)','sodium (PDV)','protein (PDV)','saturated fat (PDV)','carbohydrates (PDV)']] = df.nutrition.str.split(",",expand=True)

## Replace [] brackets after split
df['calories'] =  df['calories'].apply(lambda x: x.replace('[',''))
df['carbohydrates (PDV)'] =  df['carbohydrates (PDV)'].apply(lambda x: x.replace(']',''))

## Convert the datatypes from str to float
df[['calories','total fat (PDV)','sugar (PDV)','sodium (PDV)','protein (PDV)','saturated fat (PDV)','carbohydrates (PDV)']] = df[['calories','total fat (PDV)','sugar (PDV)','sodium (PDV)','protein (PDV)','saturated fat (PDV)','carbohydrates (PDV)']].astype('float')

## Drop unnecessary columns
df.drop(['id', 'contributor_id','submitted','tags','nutrition'], axis=1,inplace = True)

In [ ]:
df.dtypes

name                    object
minutes                  int64
n_steps                  int64
steps                   object
description             object
ingredients             object
n_ingredients            int64
calories               float64
total fat (PDV)        float64
sugar (PDV)            float64
sodium (PDV)           float64
protein (PDV)          float64
saturated fat (PDV)    float64
carbohydrates (PDV)    float64
dtype: object

In [ ]:
df.head()

,name,minutes,n_steps,steps,description,ingredients,n_ingredients,calories,total fat (PDV),sugar (PDV),sodium (PDV),protein (PDV),saturated fat (PDV),carbohydrates (PDV)
0,arriba baked winter squash mexican style,55,11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7,51.5,0.0,13.0,0.0,2.0,0.0,4.0
1,a bit different breakfast pizza,30,9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6,173.4,18.0,0.0,17.0,22.0,35.0,1.0
2,all in the kitchen chili,130,6,"['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...",13,269.8,22.0,32.0,48.0,39.0,27.0,5.0
3,alouette potatoes,45,11,['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...",11,368.1,17.0,10.0,2.0,14.0,8.0,20.0
4,amish tomato ketchup for canning,190,5,['mix all ingredients& boil for 2 1 / 2 hours ...,my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'sugar...",8,352.9,1.0,337.0,23.0,3.0,0.0,28.0


## Cleaning the ingredients column

In [ ]:
%%time
## First convert the string representation of list to list
df['ingredients'] = df['ingredients'].str.strip("[]").str.replace("'","") #.str.split(",")

CPU times: user 438 ms, sys: 72.9 ms, total: 511 ms
Wall time: 640 ms


In [ ]:
df['ingredients'].iloc[0]

'winter squash, mexican seasoning, mixed spice, honey, butter, olive oil, salt'

Explode the series containing list to a single list:

In [ ]:
ingredients = df['ingredients'].explode().str.lower().str.strip()
print(f"Around {ingredients.shape[0]} ingredients are there in the dataset")

Around 2103719 ingredients are there in the dataset


Now find the unique ingredients:

In [ ]:
unique_ingredients = ingredients.unique()
print(f"There around {unique_ingredients.shape[0]} are unique out of {ingredients.shape[0]}")

There around 14906 are unique out of 2103719


There might be some same words that mean the same but with different name.

Try with some keywords to see if it exists like search for 'egg' or 'milk'

In [ ]:
keyword = 'bread'
unique_word = ingredients[ingredients.str.contains(keyword.lower())].unique()
if len(unique_word)> 0:
    sample = ", ".join(np.random.choice(unique_word,10))
    print(f"There are around {len(unique_word)} unique ingredients containing the work 'f{keyword}'.\n\n Example:\n\n {sample}....")
else:
    print(f"No ingredients for the keyword: {keyword}")

There are around 314 unique ingredients containing the work 'fbread'.

 Example:

 parmesan flavored breadcrumbs, oatnut bread, dry rye breadcrumb, whole wheat bread cubes, italian breadcrumbs, muffuletta bread, sugar-free shortbread cookies, bread improver, gluten-free bread mix, irish soda bread....


## Sentence encoder model

In [ ]:
%%time
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.1k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

CPU times: user 3.17 s, sys: 2.35 s, total: 5.52 s
Wall time: 47.6 s


In [ ]:
model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 384, 'do_lower_case': False}) with Transformer model: MPNetModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
)

In [ ]:
df_sample = df.sample(1000).reset_index()


,index,name,minutes,n_steps,steps,...,sugar (PDV),sodium (PDV),protein (PDV),saturated fat (PDV),carbohydrates (PDV)
0,120194,layered seafood appetizer,10,8,"['mix cream cheese , mayonnaise and seasoning ...",...,3.0,15.0,30.0,56.0,1.0
1,87435,fresh fried spicy soft tacos,20,12,"['in a medium skillet , brown meat on medium-h...",...,17.0,43.0,53.0,100.0,13.0
2,86154,french chicken with balsamic vinegar crock pot,370,4,['place chicken in the bottom of the crockpot'...,...,15.0,26.0,90.0,7.0,5.0
3,30474,brownie cups,65,8,"['in a saucepan over low heat , melt the butte...",...,88.0,3.0,6.0,44.0,9.0
4,208541,tangy tomato rubbed steak,185,8,['pierce steak all over with fork and season w...,...,0.0,18.0,48.0,19.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
995,39031,cheddar spirals,170,6,"['cook pasta according to package directions',...",...,4.0,20.0,33.0,85.0,10.0
996,111715,iraqi pink rice plow ahmar,45,11,"['heat olive oil', 'add onion', 'fry until tra...",...,12.0,15.0,8.0,2.0,14.0
997,39010,cheddar potatoes and onions,45,11,"['preheat oven to 350f', 'pierce potatoes with...",...,8.0,16.0,12.0,15.0,7.0
998,105449,hershey s st patrick s day parfaits,15,10,['combine marshmallows and milk in a saucepan ...,...,147.0,2.0,7.0,95.0,15.0


In [ ]:
%%time

## encode all the ingredients:
#embedding = model.encode(df_sample['ingredients'].values)
embedding = map(model.encode, df['ingredients'].values)

CPU times: user 109 µs, sys: 0 ns, total: 109 µs
Wall time: 96.3 µs


In [ ]:
%%time
embedding = list(embedding)

In [ ]:
def cosine_similarity(vec1,vec2):
    '''
    Returns the cosine similarity between two vectors of n dimension
    '''
    a = np.array(vec1).ravel()
    b = np.array(vec2).ravel()
    denom = np.sqrt(np.sum(np.square(vec1))) * np.sqrt(np.sum(np.square(vec2)))
    return np.round(np.dot(a,b) / denom * 100, 2)

In [ ]:
recipe = 'layered seafood appetizer'
index = df_sample.loc[df_sample['name']==recipe, "ingredients"].index
if len(index)>0 :
    ingredients = df_sample.iloc[index[0]]['ingredients']
    # df_sample['ingredients'].apply(lambda x: cosine_similarity())

index, ingredients

(Int64Index([0], dtype='int64'),
 'cream cheese, mayonnaise, old bay seasoning, green onion, tomatoes, lump crabmeat, cheddar cheese, crackers')

In [ ]:
%%time
vec1 = embedding[index]
def cs(x):
    return cosine_similarity(vec1,x)
sim = sorted(list(map(cs, embedding)), reverse=True)

CPU times: user 43.3 ms, sys: 983 µs, total: 44.3 ms
Wall time: 43.8 ms


In [ ]:
sim

[100.0,
 88.91,
 87.77,
 84.97,
 84.71,
 84.53,
 84.42,
 84.27,
 84.24,
 84.16,
 83.93,
 83.82,
 83.78,
 83.74,
 83.63,
 83.59,
 83.15,
 83.09,
 83.06,
 83.05,
 82.59,
 82.52,
 82.45,
 82.39,
 82.31,
 82.28,
 82.23,
 82.2,
 82.19,
 82.16,
 82.14,
 81.81,
 81.79,
 81.78,
 81.78,
 81.72,
 81.57,
 81.54,
 81.36,
 81.26,
 81.13,
 81.12,
 81.01,
 80.9,
 80.83,
 80.78,
 80.63,
 80.55,
 80.48,
 80.43,
 80.4,
 80.39,
 80.37,
 80.35,
 80.34,
 80.28,
 80.25,
 80.24,
 80.19,
 80.19,
 80.13,
 80.12,
 80.07,
 80.07,
 80.05,
 80.03,
 79.98,
 79.94,
 79.93,
 79.82,
 79.7,
 79.68,
 79.67,
 79.65,
 79.65,
 79.56,
 79.53,
 79.53,
 79.46,
 79.42,
 79.4,
 79.38,
 79.34,
 79.33,
 79.32,
 79.29,
 79.28,
 79.26,
 79.19,
 79.07,
 79.05,
 79.04,
 79.02,
 79.01,
 78.95,
 78.93,
 78.84,
 78.83,
 78.76,
 78.68,
 78.65,
 78.62,
 78.59,
 78.57,
 78.51,
 78.5,
 78.44,
 78.36,
 78.32,
 78.3,
 78.28,
 78.28,
 78.26,
 78.23,
 78.2,
 78.17,
 78.1,
 78.09,
 78.04,
 78.0,
 77.98,
 77.97,
 77.96,
 77.92,
 77.82,
 77.74,
 7